In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as K
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dense

import time
from alive_progress import alive_bar

In [3]:
import sys
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.3+
    count = len(it)
    def show(j):
        x = int(size*j/count)
        print("{}[{}{}] {}/{}".format(prefix, "#"*x, "."*(size-x), j, count), 
                end='\r', file=out, flush=True)
    show(0)
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

In [4]:
def create_model(layers = [64,64,128,128], board_size = 8):
    model = Sequential()
    model.add(Conv2D(layers[0], 3, padding='same', input_shape = (board_size, board_size, 2)))
    for layer in layers[1:]:
        model.add(Conv2D(layer, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(units = 128))
    model.add(Dense(units = 64, activation = 'sigmoid'))
    return model

In [5]:
def convert_input(board, player_icon, enemy_icon, board_size = 8):
    new_board1 = list(map(lambda l: list(map(lambda x: 1 if x == player_icon else 0, l)), board)) 
    new_board2 = list(map(lambda l: list(map(lambda x: 1 if x == enemy_icon else 0, l)), board))
    new_board = np.zeros((board_size, board_size, 2))
    new_board[:,:,0] = new_board1
    new_board[:,:,1] = new_board2
    return new_board

In [6]:
model = create_model(board_size = 3)
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.predict(convert_input([
    ['X', 'X', 'O'],
    ['O', '*', 'O'],
    ['*', '*', 'X']
], 'X', 'O', 3).reshape(1,3,3,2))

1/1 [==============================] - 0s 428ms/step


array([[0.50389653, 0.5098615 , 0.49962384, 0.4903542 , 0.502917  ,
        0.49920473, 0.50628895, 0.5089431 , 0.5103953 , 0.47509778,
        0.5201425 , 0.5345406 , 0.48984388, 0.4608071 , 0.5074218 ,
        0.4845902 , 0.48339906, 0.47969335, 0.51080436, 0.50895095,
        0.5065337 , 0.4588479 , 0.51768637, 0.5176378 , 0.52039295,
        0.50969744, 0.4836762 , 0.49425548, 0.47880992, 0.5009057 ,
        0.5192175 , 0.49199763, 0.52867293, 0.50136703, 0.4981668 ,
        0.5067854 , 0.52388537, 0.48446473, 0.5121682 , 0.46821427,
        0.5054819 , 0.50964844, 0.5132291 , 0.47347873, 0.4856493 ,
        0.5229611 , 0.49105456, 0.52151585, 0.49567583, 0.54441947,
        0.5381513 , 0.5271817 , 0.5149442 , 0.5025221 , 0.49250275,
        0.49909014, 0.49506864, 0.4791814 , 0.5206602 , 0.5049606 ,
        0.499244  , 0.500363  , 0.4974597 , 0.5124634 ]], dtype=float32)

In [7]:
df = pd.read_csv('othello_dataset.csv')

# Remove the first and second column
df = df.iloc[:,2:]

df

,game_moves
0,f5d6c4d3e6f4e3f6c5b4e7f3c6d7b5a5c3b3g5h5g4h4e2...
1,d3c5f6f5e6e3d6f7b6d7e2c6d8c4e7c3f4c8c7d2d1a6b3...
2,f5d6c4d3c3f4e6b3e2c5c7f6g5e3e7g6g4h4f3g3f2h6h5...
3,f5d6c5b4c3e3e2f4f3g4e6g5f6f7e7g3h6c4d3h5g6h7b3...
4,f5d6c4d3c3f4c5b4g3c6b5f6e2g4e3e6b3a5d7e7f3a3h4...
...,...
25652,c4e3f5c5c3e6f4g4d6d3c6b4d2c2f3f6b3b5h4g3h3f2e2...
25653,c4e3f6e6f5c5c3c6d3d2f3b3b4c2b5d6e2f4f2e1c1g5g4...
25654,f5d6c4d3c3f4f6f3e6e7d7c5b6b5c6b4e3e8g5c2f8d2d1...
25655,f5d6c3f4f6d3c4f3e6c6c5e7e3e2f7g5g6f8h5h6h7b6b5...


In [8]:
def board_to_columns(board):
    return {f'b{k}_{i}_{j}': board[i][j][k] for i in range(len(board)) for j in range(len(board[0])) for k in range(len(board[0][0]))}

In [78]:
def check_valid_move(board, move, player_icon, enemy_icon, empty_icon, board_size = 8):
    for i in range(-1, 2):
        for j in range(-1, 2):
            if(i != 0 or j != 0):
                # Check if the move is valid
                if(move[0] + i >= 0 and move[0] + i < board_size and move[1] + j >= 0 and move[1] + j < board_size):
                    if(board[move[0] + i][move[1] + j] == enemy_icon):
                        # Check if there is a player icon in the direction
                        k = 2
                        while(move[0] + i*k >= 0 and move[0] + i*k < board_size and move[1] + j*k >= 0 and move[1] + j*k < board_size):
                            if(board[move[0] + i*k][move[1] + j*k] == player_icon):
                                return True
                            if(board[move[0] + i*k][move[1] + j*k] == empty_icon):
                                break
                            k += 1
    return False

In [83]:
def generate_boards(board):
    for i in range(4):
        flipped = np.fliplr(board)
        yield np.rot90(board, i)
        yield np.rot90(flipped, i)

In [76]:
def print_board(board, num_round):
    print('='*20)
    print(f'Round {num_round}')
    for i in range(len(board)):
        for j in range(len(board[0])):
            print(board[i][j], end = ' ')
        print()
    print('='*20)

In [87]:
# Create empty dict for new dataset
df_formatted_dict = {}

def create_rows(MAX_ITER = -1):
    # Players icons
    player_icon = 'X'
    enemy_icon = 'O'
    empty_icon = '*'

    if(MAX_ITER == -1): MAX_ITER = len(df)

    # For each row in the dataset
    for i in range(MAX_ITER):
        # Get moves
        moves_str = df.iloc[i, 0]
        moves = list(map(
            lambda x: (ord(x[0]) - ord('a'), int(x[1]) - 1),
            [moves_str[k:k+2] for k in range(0, len(moves_str), 2)]
        ))


        # Create empty board
        board = [[empty_icon for i in range(8)] for j in range(8)]
        board[3][3] = enemy_icon 
        board[4][4] = enemy_icon
        board[3][4] = player_icon
        board[4][3] = player_icon

        num_round = 1

        # For each move
        for move in moves:
            # If move in invalid, change icons back
            if(not check_valid_move(board, move, player_icon, enemy_icon, empty_icon)):
                player_icon, enemy_icon = enemy_icon, player_icon

            # Get move in number format
            move_num = move[0] * 8 + move[1]

            # Add boards to df_formatted_dict
            for board_rot in generate_boards(board):
                df_formatted_dict[len(df_formatted_dict)] = {
                    **board_to_columns(convert_input(board_rot, player_icon, enemy_icon)),
                    'move': move_num
                }
                df_formatted_dict[len(df_formatted_dict)] = {
                    **board_to_columns(convert_input(board_rot, enemy_icon, player_icon)),
                    'move': move_num
                }

            # Update board
            board[move[0]][move[1]] = player_icon
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if(i == 0 and j == 0): continue
                    # Check if the move is valid
                    if(move[0] + i >= 0 and move[0] + i < 8 and move[1] + j >= 0 and move[1] + j < 8):
                        if(board[move[0] + i][move[1] + j] == enemy_icon):
                            # Check if there is a player icon in the direction
                            k = 2
                            while(move[0] + i*k >= 0 and move[0] + i*k < 8 and move[1] + j*k >= 0 and move[1] + j*k < 8):
                                if(board[move[0] + i*k][move[1] + j*k] == player_icon):
                                    for l in range(1, k):
                                        board[move[0] + i*l][move[1] + j*l] = player_icon
                                    break
                                if(board[move[0] + i*k][move[1] + j*k] == empty_icon):
                                    break
                                k += 1
            player_icon, enemy_icon = enemy_icon, player_icon
            # print_board(board, num_round)
            num_round += 1

create_rows(10)

In [85]:
# Create new dataset
df_formatted = pd.DataFrame.from_dict(df_formatted_dict, orient='index')

# Remove duplicates
df_formatted = df_formatted.drop_duplicates()

df_formatted

,b0_0_0,b1_0_0,b0_0_1,b1_0_1,b0_0_2,b1_0_2,b0_0_3,b1_0_3,b0_0_4,b1_0_4,...,b1_7_3,b0_7_4,b1_7_4,b0_7_5,b1_7_5,b0_7_6,b1_7_6,b0_7_7,b1_7_7,move
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,56
956,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,56
957,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,56
958,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,56


In [56]:
df_formatted_dict[0]

{'b0_0_0': 0.0,
 'b1_0_0': 0.0,
 'b0_0_1': 0.0,
 'b1_0_1': 0.0,
 'b0_0_2': 0.0,
 'b1_0_2': 0.0,
 'b0_0_3': 0.0,
 'b1_0_3': 0.0,
 'b0_0_4': 0.0,
 'b1_0_4': 0.0,
 'b0_0_5': 0.0,
 'b1_0_5': 0.0,
 'b0_0_6': 0.0,
 'b1_0_6': 0.0,
 'b0_0_7': 0.0,
 'b1_0_7': 0.0,
 'b0_1_0': 0.0,
 'b1_1_0': 0.0,
 'b0_1_1': 0.0,
 'b1_1_1': 0.0,
 'b0_1_2': 0.0,
 'b1_1_2': 0.0,
 'b0_1_3': 0.0,
 'b1_1_3': 0.0,
 'b0_1_4': 0.0,
 'b1_1_4': 0.0,
 'b0_1_5': 0.0,
 'b1_1_5': 0.0,
 'b0_1_6': 0.0,
 'b1_1_6': 0.0,
 'b0_1_7': 0.0,
 'b1_1_7': 0.0,
 'b0_2_0': 0.0,
 'b1_2_0': 0.0,
 'b0_2_1': 0.0,
 'b1_2_1': 0.0,
 'b0_2_2': 0.0,
 'b1_2_2': 0.0,
 'b0_2_3': 0.0,
 'b1_2_3': 0.0,
 'b0_2_4': 0.0,
 'b1_2_4': 0.0,
 'b0_2_5': 0.0,
 'b1_2_5': 0.0,
 'b0_2_6': 0.0,
 'b1_2_6': 0.0,
 'b0_2_7': 0.0,
 'b1_2_7': 0.0,
 'b0_3_0': 0.0,
 'b1_3_0': 0.0,
 'b0_3_1': 0.0,
 'b1_3_1': 0.0,
 'b0_3_2': 0.0,
 'b1_3_2': 0.0,
 'b0_3_3': 0.0,
 'b1_3_3': 1.0,
 'b0_3_4': 1.0,
 'b1_3_4': 0.0,
 'b0_3_5': 0.0,
 'b1_3_5': 0.0,
 'b0_3_6': 0.0,
 'b1_3_6': 0.0,
 'b0_3_7